In [2]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 374 (delta 85), reused 52 (delta 48), pack-reused 269
Receiving objects: 100% (374/374), 23.11 MiB | 27.19 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

%run "./Aerospace-Project/Standard Filters/DiffKf.ipynb"
%run "./Aerospace-Project/Standard Filters/KF.ipynb"

OPTIMIZATION CLARIFICATIONS: \\
TO COMPUTE THE COVARIANCE OF CKF AND DKF THE FILTER IS BEING RUN EVERYTIME RATHER THAN USING STEADY STATE COVARIANCE OF CKF. \\
THIS CHANGES THE BHATTACHARYA DISTANCE COMPUTED FOR THE "OPTIMAL WEIGHTS".


Simulated Annealing

Particle Swarm Optimization

Parallel Tempering

STOCHASTIC TUNNELING:
Bhattacharya Distance computation seems to be incorrect.

In [41]:


from scipy.optimize import minimize

# Bhattacharyya distance function
def bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2):
    Sigma = (Sigma1 + Sigma2) / 2
    inv_Sigma = np.linalg.inv(Sigma)
    term1 = 1/8 * np.dot(np.dot((mu1 - mu2).T, inv_Sigma), (mu1 - mu2))
    term2 = 1/2 * np.log(np.linalg.det(Sigma) / np.sqrt(np.linalg.det(Sigma1) * np.linalg.det(Sigma2)))
    return term1 + term2

# Function to run filters and return covariances
def run_filters(W):
    print(W)
    dt = 10

    # define C
    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])
    C = C_adj * np.reshape(W, (5, 5))
    C_unweighted = np.array([[1 if x != 0 else 0 for x in row] for row in C])
    num_stns = len(C[0])

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0],[0,0,1,0]])

    dkf_state_size = len(A)
    dkf_measure_size = len(H)

    q = 0.001
    Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
    R = np.array([[4,0],[0,4]])

    A_kf = np.kron(np.eye(num_stns), A)
    H_kf = np.kron(np.eye(num_stns), H)
    Q_kf = np.kron(np.eye(num_stns), Q)
    R_kf = np.kron(np.eye(num_stns), R)

    kf_state_size = A_kf.shape[0]
    kf_measure_size = R_kf.shape[0]

    F = [A for _ in range(num_stns)]
    G = [np.eye(A.shape[0]) for _ in range(num_stns)]
    H_dkf = [H for _ in range(num_stns)]

    Q_dkf = [Q for _ in range(num_stns)]
    R_dkf = [R for _ in range(num_stns)]

    procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
    measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

    measure_kf_to_dkf  = lambda z: [np.array([z[H.shape[0]*i + j] for j in range(H.shape[0])]) for i in range(num_stns)]
    state_kf_to_dkf = lambda z: [np.array([z[A.shape[0]*i + j] for j in range(A.shape[0])]) for i in range(num_stns)]

    # True Initial
    x0_kf = np.array([[np.random.normal(0, np.sqrt(Q_kf[i, i])) for i in range(kf_state_size)]]).T

    # Initial Estimate
    x_kf = np.array([[np.random.normal(0, 5) for i in range(kf_state_size)]]).T
    x_dkf = state_kf_to_dkf(x_kf)

    P_kf = 10 * np.copy(Q_kf)
    P_dkf = [10 * np.copy(Q) for _ in range(num_stns)]

    kf = KalmanFilter(A=A_kf, H=H_kf, Q=Q_kf, R=R_kf, P=P_kf, x0=x0_kf)
    dkf = DiffKF(C, F, G, H_dkf, R_dkf, Q_dkf, x_dkf, P_dkf)

    iters = 60

    truth = np.zeros((iters + 1, kf_state_size, 1))
    truth[0] = x0_kf

    measurements = np.zeros((iters + 1, kf_measure_size, 1))
    measurements[0] = (H_kf @ x0_kf) + measure_noise_kf()

    predictions_kf = np.zeros((iters, kf_state_size, 1))
    predictions_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    errors_kf = np.zeros((iters, kf_state_size, 1))
    errors_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    P_hist_kf = np.zeros((iters, kf_state_size, kf_state_size))
    P_hist_dkf = np.zeros((iters, num_stns, A.shape[0], A.shape[0]))
    full_system_P_hist = np.zeros((iters, kf_state_size, kf_state_size))
    prev_cov = np.block([[np.zeros(P_dkf[0].shape) if i != j else dkf.nodes[i].P for j in range(num_stns)] for i in range(num_stns)])

    for i in range(iters):
        kf.update(measurements[i])
        dkf.update(measure_kf_to_dkf(measurements[i]))

        predictions_dkf[i] = [dkf.nodes[j].x for j in range(num_stns)]
        errors_dkf[i] = [dkf.nodes[j].x - state_kf_to_dkf(truth[i])[j] for j in range(num_stns)]
        station_covs = [dkf.nodes[j].P for j in range(num_stns)]
        P_hist_dkf[i] = station_covs

        prev_cov = get_diff_cov(prev_cov, station_covs, dkf, num_stns, A, H, Q, R, C, C_unweighted, G)
        full_system_P_hist[i] = prev_cov

        predictions_kf[i] = kf.x
        errors_kf[i] = kf.x - truth[i]
        P_hist_kf[i] = kf.P

        kf.predict()
        dkf.predict()

        truth[i + 1] = A_kf @ x0_kf + procc_noise_kf()
        measurements[i + 1] = H_kf @ truth[i + 1] + measure_noise_kf()

    return (P_hist_kf[40], full_system_P_hist[40])

# Function to get diffusion covariance
def get_diff_cov(prev_cov, Station_cov, dkf, num_stns, A, H, Q, R, C, C_unweighted, G):
    S = lambda i: np.sum([node.H.T @ np.linalg.inv(node.R) @ node.H for node in dkf.nodes[i].nbhrs], axis=0)

    S_full = np.block([[np.zeros(A.shape) if i != j else S(j) for j in range(num_stns)] for i in range(num_stns)])
    H_full = np.kron(np.eye(num_stns), H)
    P_full = np.block([[np.zeros(Station_cov[0].shape) if i != j else Station_cov[j] for j in range(num_stns)] for i in range(num_stns)])
    R_full = np.kron(np.eye(num_stns), R)

    C_full = np.kron(C, np.eye(A.shape[0]))
    A_full = np.kron(C_unweighted, np.eye(A.shape[0]))

    F_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), A)
    G_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), G[0])
    D_i = C_full.T @ P_full @ A_full.T @ H_full.T @ np.linalg.inv(R_full)

    term1 = (F_i @ prev_cov @ F_i.T)
    term2 = G_i @ np.kron(np.ones((num_stns, num_stns)), Q) @ G_i.T
    term3 = D_i @ R_full @ D_i.T

    return term1 + term2 + term3

# Define the cost function for optimization
def cost_func(diffusion_weights):
    dt = 10
    num_stns = 5

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
    q = 0.001
    Q = q * np.array([[(dt**3) / 3, (dt**2) / 2, 0, 0],
                      [(dt**2) / 2, dt, 0, 0],
                      [0, 0, (dt**3) / 3, (dt**2) / 2],
                      [0, 0, (dt**2) / 2, dt]])
    R = np.array([[4, 0], [0, 4]])

    # size of state and measurement vectors
    kf_state_size = A.shape[0] * num_stns
    kf_measure_size = H.shape[0] * num_stns

    # initialize the means to 0
    mu1 = np.zeros(kf_state_size)
    mu2 = np.zeros(kf_state_size)

    # run filters to get the sigma tuple
    sigmas = run_filters(diffusion_weights)
    Sigma1 = sigmas[0] # state covariance of centralized KF
    Sigma2 = sigmas[1] # state covariance of diffusion KF
    return bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2)

# Stochastic Tunneling optimization
def stochastic_tunneling(cost_func, initial_state, C_adj, num_iterations, rho=1.0):
    best_state = initial_state.copy()
    best_energy = cost_func(best_state)
    current_state = initial_state.copy()
    current_energy = best_energy

    for iteration in range(num_iterations):
        # Propose a new state by perturbing the current state
        new_state = current_state.copy()
        idx = np.random.choice(np.where(C_adj.flatten() == 1)[0])
        new_state[idx] += np.random.normal(0, 0.1)

        # Ensure constraints
        new_state[new_state < 0] = 0
        for i in range(0, len(new_state), 5):
            row = new_state[i:i + 5] * C_adj[i//5]
            row_sum = np.sum(row)
            if row_sum != 0:
                new_state[i:i + 5] = (row / row_sum) * C_adj[i//5]

        # Calculate the new energy
        new_energy = cost_func(new_state)

        # Apply tunneling transformation
        delta_energy = new_energy - current_energy
        if delta_energy < 0 or np.random.rand() < np.exp(-rho * delta_energy):
            current_state = new_state
            current_energy = new_energy

            # Update the best state if the new state is better
            if new_energy < best_energy:
                best_state = new_state
                best_energy = new_energy

        print(f"Iteration {iteration+1}/{num_iterations}, Current Energy: {current_energy}, Best Energy: {best_energy}")

    return best_state, best_energy

# Run the optimization
def run_optimize():
    C = np.array([[0.34, 0.33, 0, 0, 0.33],
                  [0.33, 0.34, 0.33, 0, 0],
                  [0, 0.33, 0.34, 0.33, 0],
                  [0, 0, 0.33, 0.34, 0.33],
                  [0.33, 0, 0, 0.33, 0.34]])

    # C = np.array([[0.52181377, 0.03155906, 0.        , 0.        , 0.44662717],
    #           [0.        , 0.47536002, 0.52463998, 0.        , 0.        ],
    #           [0.        , 0.80184446, 0.06769371, 0.13046183, 0.        ],
    #           [0.        , 0.        , 0.        , 0.        , 1.        ],
    #           [0.13487005, 0.        , 0.        , 0.29567589, 0.56945406]])

    # C = np.array([[0.52296034, 0.        , 0.        , 0.        , 0.47703966],
    #           [0.        , 0.35764041, 0.64235959, 0.        , 0.        ],
    #           [0.        , 1.        , 0.        , 0.        , 0.        ],
    #           [0.        , 0.        , 0.        , 0.        , 1.        ],
    #           [0.00850923, 0.        , 0.        , 0.75734611, 0.23414466]])

    # C = np.array([[0.97634767, 0.00299567, 0.        , 0.        , 0.02065666],
    #           [0.        , 0.        , 1.        , 0.        , 0.        ],
    #           [0.        , 1.        , 0.        , 0.        , 0.        ],
    #           [0.        , 0.        , 0.00115229, 0.03434087, 0.96450684],
    #           [0.        , 0.        , 0.        , 1.        , 0.        ]])



    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])

    # Flatten C matrix to use as initial weights
    initial_state = C.flatten()

    num_iterations = 100
    rho = 10.0
    best_state, best_energy = stochastic_tunneling(cost_func, initial_state, C_adj, num_iterations, rho)

    # Checks if each row in the final best weights matrix sums to 1
    best_weights_matrix = np.reshape(best_state, (5, 5))
    row_sums = np.sum(best_weights_matrix, axis=1)
    print(f'Best weights matrix:\n{best_weights_matrix}')
    print(f'Row sums: {row_sums}')
    print(f'Best Bhattacharyya distance: {best_energy}')

run_optimize()


[0.34 0.33 0.   0.   0.33 0.33 0.34 0.33 0.   0.   0.   0.33 0.34 0.33
 0.   0.   0.   0.33 0.34 0.33 0.33 0.   0.   0.33 0.34]
[0.40272078 0.20640316 0.         0.         0.39087606 0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.33       0.         0.         0.33
 0.34      ]
Iteration 1/100, Current Energy: 22.148695157258093, Best Energy: 22.148695157258093
[0.34415178 0.17638527 0.         0.         0.47946295 0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.33       0.         0.         0.33
 0.34      ]
Iteration 2/100, Current Energy: 21.535323919864997, Best Energy: 21.535323919864997
[0.34415178 0.17638527 0.         0.         0.47946295 0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.31370267

ANT COLONY OPTIMIZATION: \\
Pros - Does not require a intial guess \\
Cons - very slow \\

In [42]:


# Bhattacharyya distance function
def bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2):
    Sigma = (Sigma1 + Sigma2) / 2
    inv_Sigma = np.linalg.inv(Sigma)
    term1 = 1/8 * np.dot(np.dot((mu1 - mu2).T, inv_Sigma), (mu1 - mu2))
    term2 = 1/2 * np.log(np.linalg.det(Sigma) / np.sqrt(np.linalg.det(Sigma1) * np.linalg.det(Sigma2)))
    return term1 + term2

# Function to run filters and return covariances
def run_filters(W):
    print(W)
    dt = 10

    # define C
    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])
    C = C_adj * np.reshape(W, (5, 5))
    C_unweighted = np.array([[1 if x != 0 else 0 for x in row] for row in C])
    num_stns = len(C[0])

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0],[0,0,1,0]])

    dkf_state_size = len(A)
    dkf_measure_size = len(H)

    q = 0.001
    Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
    R = np.array([[4,0],[0,4]])

    A_kf = np.kron(np.eye(num_stns), A)
    H_kf = np.kron(np.eye(num_stns), H)
    Q_kf = np.kron(np.eye(num_stns), Q)
    R_kf = np.kron(np.eye(num_stns), R)

    kf_state_size = A_kf.shape[0]
    kf_measure_size = R_kf.shape[0]

    F = [A for _ in range(num_stns)]
    G = [np.eye(A.shape[0]) for _ in range(num_stns)]
    H_dkf = [H for _ in range(num_stns)]

    Q_dkf = [Q for _ in range(num_stns)]
    R_dkf = [R for _ in range(num_stns)]

    procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
    measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

    measure_kf_to_dkf  = lambda z: [np.array([z[H.shape[0]*i + j] for j in range(H.shape[0])]) for i in range(num_stns)]
    state_kf_to_dkf = lambda z: [np.array([z[A.shape[0]*i + j] for j in range(A.shape[0])]) for i in range(num_stns)]

    # True Initial
    x0_kf = np.array([[np.random.normal(0, np.sqrt(Q_kf[i, i])) for i in range(kf_state_size)]]).T

    # Initial Estimate
    x_kf = np.array([[np.random.normal(0, 5) for i in range(kf_state_size)]]).T
    x_dkf = state_kf_to_dkf(x_kf)

    P_kf = 10 * np.copy(Q_kf)
    P_dkf = [10 * np.copy(Q) for _ in range(num_stns)]

    kf = KalmanFilter(A=A_kf, H=H_kf, Q=Q_kf, R=R_kf, P=P_kf, x0=x0_kf)
    dkf = DiffKF(C, F, G, H_dkf, R_dkf, Q_dkf, x_dkf, P_dkf)

    iters = 60

    truth = np.zeros((iters + 1, kf_state_size, 1))
    truth[0] = x0_kf

    measurements = np.zeros((iters + 1, kf_measure_size, 1))
    measurements[0] = (H_kf @ x0_kf) + measure_noise_kf()

    predictions_kf = np.zeros((iters, kf_state_size, 1))
    predictions_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    errors_kf = np.zeros((iters, kf_state_size, 1))
    errors_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    P_hist_kf = np.zeros((iters, kf_state_size, kf_state_size))
    P_hist_dkf = np.zeros((iters, num_stns, A.shape[0], A.shape[0]))
    full_system_P_hist = np.zeros((iters, kf_state_size, kf_state_size))
    prev_cov = np.block([[np.zeros(P_dkf[0].shape) if i != j else dkf.nodes[i].P for j in range(num_stns)] for i in range(num_stns)])

    for i in range(iters):
        kf.update(measurements[i])
        dkf.update(measure_kf_to_dkf(measurements[i]))

        predictions_dkf[i] = [dkf.nodes[j].x for j in range(num_stns)]
        errors_dkf[i] = [dkf.nodes[j].x - state_kf_to_dkf(truth[i])[j] for j in range(num_stns)]
        station_covs = [dkf.nodes[j].P for j in range(num_stns)]
        P_hist_dkf[i] = station_covs

        prev_cov = get_diff_cov(prev_cov, station_covs, dkf, num_stns, A, H, Q, R, C, C_unweighted, G)
        full_system_P_hist[i] = prev_cov

        predictions_kf[i] = kf.x
        errors_kf[i] = kf.x - truth[i]
        P_hist_kf[i] = kf.P

        kf.predict()
        dkf.predict()

        truth[i + 1] = A_kf @ x0_kf + procc_noise_kf()
        measurements[i + 1] = H_kf @ truth[i + 1] + measure_noise_kf()

    return (P_hist_kf[40], full_system_P_hist[40])

# Function to get diffusion covariance
def get_diff_cov(prev_cov, Station_cov, dkf, num_stns, A, H, Q, R, C, C_unweighted, G):
    S = lambda i: np.sum([node.H.T @ np.linalg.inv(node.R) @ node.H for node in dkf.nodes[i].nbhrs], axis=0)

    S_full = np.block([[np.zeros(A.shape) if i != j else S(j) for j in range(num_stns)] for i in range(num_stns)])
    H_full = np.kron(np.eye(num_stns), H)
    P_full = np.block([[np.zeros(Station_cov[0].shape) if i != j else Station_cov[j] for j in range(num_stns)] for i in range(num_stns)])
    R_full = np.kron(np.eye(num_stns), R)

    C_full = np.kron(C, np.eye(A.shape[0]))
    A_full = np.kron(C_unweighted, np.eye(A.shape[0]))

    F_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), A)
    G_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), G[0])
    D_i = C_full.T @ P_full @ A_full.T @ H_full.T @ np.linalg.inv(R_full)

    term1 = (F_i @ prev_cov @ F_i.T)
    term2 = G_i @ np.kron(np.ones((num_stns, num_stns)), Q) @ G_i.T
    term3 = D_i @ R_full @ D_i.T

    return term1 + term2 + term3


# Define the cost function for optimization
def cost_func(diffusion_weights):
    dt = 10
    num_stns = 5

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
    q = 0.001
    Q = q * np.array([[(dt**3) / 3, (dt**2) / 2, 0, 0],
                      [(dt**2) / 2, dt, 0, 0],
                      [0, 0, (dt**3) / 3, (dt**2) / 2],
                      [0, 0, (dt**2) / 2, dt]])
    R = np.array([[4, 0], [0, 4]])

    # size of state and measurement vectors
    kf_state_size = A.shape[0] * num_stns
    kf_measure_size = H.shape[0] * num_stns

    # initialize the means to 0
    mu1 = np.zeros(kf_state_size)
    mu2 = np.zeros(kf_state_size)

    # run filters to get the sigma tuple
    sigmas = run_filters(diffusion_weights)
    Sigma1 = sigmas[0] # state covariance of centralized KF
    Sigma2 = sigmas[1] # state covariance of diffusion KF
    return bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2)

class AntColonyOptimization:
    def __init__(self, cost_func, num_ants, num_iterations, C_adj, decay=0.95, alpha=1, beta=1):
        self.cost_func = cost_func
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.C_adj = C_adj
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.pheromone = np.ones(C_adj.shape)
        self.best_cost = float('inf')
        self.best_solution = None

    def initialize_ants(self):
        return [self.initialize_solution() for _ in range(self.num_ants)]

    def initialize_solution(self):
        solution = np.random.rand(*self.C_adj.shape) * self.C_adj
        for i in range(solution.shape[0]):
            row_sum = np.sum(solution[i])
            if row_sum > 0:
                solution[i] /= row_sum
        return solution.flatten()

    def evaluate_solution(self, solution):
        return self.cost_func(solution)

    def update_pheromone(self, solutions, costs):
        for i in range(len(solutions)):
            solution = solutions[i]
            cost = costs[i]
            pheromone_delta = 1 / (cost + 1e-10)
            reshaped_solution = solution.reshape(self.pheromone.shape)
            self.pheromone += pheromone_delta * reshaped_solution

    def evaporate_pheromone(self):
        self.pheromone *= self.decay

    def run(self):
        for iteration in range(self.num_iterations):
            solutions = self.initialize_ants()
            costs = [self.evaluate_solution(solution) for solution in solutions]

            for i in range(len(solutions)):
                if costs[i] < self.best_cost:
                    self.best_cost = costs[i]
                    self.best_solution = solutions[i]

            self.update_pheromone(solutions, costs)
            self.evaporate_pheromone()
            print(f"Iteration {iteration+1}/{self.num_iterations}, Best Cost: {self.best_cost}")

        return self.best_solution, self.best_cost

# Run the optimization
def run_optimize():
    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])

    num_ants = 10
    num_iterations = 50
    aco = AntColonyOptimization(cost_func, num_ants, num_iterations, C_adj)
    best_solution, best_cost = aco.run()

    best_weights_matrix = np.reshape(best_solution, (5, 5))
    row_sums = np.sum(best_weights_matrix, axis=1)
    print(f'Best weights matrix:\n{best_weights_matrix}')
    print(f'Row sums: {row_sums}')
    print(f'Best Bhattacharyya distance: {best_cost}')

run_optimize()


[0.39203977 0.43018711 0.         0.         0.17777313 0.54394413
 0.29855841 0.15749746 0.         0.         0.         0.08719277
 0.49523148 0.41757576 0.         0.         0.         0.34350915
 0.2046284  0.45186246 0.57803605 0.         0.         0.38253123
 0.03943272]
[0.61763965 0.25709005 0.         0.         0.1252703  0.15061007
 0.8123398  0.03705013 0.         0.         0.         0.57535121
 0.14761812 0.27703068 0.         0.         0.         0.37074515
 0.27600341 0.35325145 0.54386671 0.         0.         0.04776126
 0.40837203]
[0.35809566 0.31877395 0.         0.         0.32313039 0.22687588
 0.65474683 0.11837729 0.         0.         0.         0.2839775
 0.50233712 0.21368538 0.         0.         0.         0.13254316
 0.22893687 0.63851997 0.29645001 0.         0.         0.34588479
 0.3576652 ]
[0.24710994 0.43156282 0.         0.         0.32132725 0.25356612
 0.42988986 0.31654402 0.         0.         0.         0.29444888
 0.30535548 0.40019564 0